<span type="title">Stand Action and EL</span> | <span type="update">2018-09-23</span> - Version <span type="version">1.1</span>
    
    
<span type="intro"><p class="card-text">本章在标准 Servlet 和 JSP 规范的基础上介绍简化开发流程的标准动作：用于 Beans 装卸和属性获取、包含、派发、用于获取映射和 Beans 数据并支持简单算术运算的 Expression Language 表达式语言、Taglib 对于 EL 提供的函数调用支持：自定义 Taglib 和 xld 以实现函数调用。</p></span>

# 1. 标准动作

标准动作指的是在 JSP 文件中，那些 `<jsp:xxx>` 包裹着的代码。这很像（或者说压根就是） XML 的文件协议。其中 xxx 部分规定了一些可供使用的标签，其中最有用的时： useBean, getProperty, setProperty。很显然，这里的目的是用于在 JSP 文件中构造以及卸载Bean。

## 1.1 使用 Beans

对于卸载Bean（使用Bean）而言，常见的流程是： 用户请求 -> 找到对应的 Servlet -> Servlet 将 Request 的属性中添加一个名为 id，类型为 class，但是转型为 type 的 Java Beans。-> Servlet 分发请求到 JSP 文件， JSP 文件获取并显示 Beans 的某个属性 property。

```java
req.setAttribute("user",new Student("Corkine",22,null));
req.getRequestDispatcher("/hello.jsp").forward(req,resp);
```

JSP 文件中使用 Beans 很简单，通过脚本 `request.getAttritube("id").getProperty()` 即可。但是，最佳实践是，不使用脚本语言，因为脚本往往带来难以维护的代码。使用标准动作可以完成这一调用和使用：

```jsp
<jsp:useBean id="user" class="com.mazhangjing.model.Student" type="com.mazhangjing.model.Person" scope="request" />

<jsp:setProperty name="user" property="address" value="Central China Normal University"/>
<h1>Hello, <jsp:getProperty name="user" property="name" /></h1>
<p>Welcome to my Website. You came from <jsp:getProperty name="user" property="address" /></p>
```

如上所示，useBean 动作从 request 请求域中使用 id = user 获取了类型为 Student 的对象，然后将其转型为 Person，相当于 `Person user = （Student）request.getAttritube("user")`。getProperty 动作使用这个变量名称作为 name，从中获取 name 这个属性，相当于 `user.getName()` 这个调用。

下面介绍一下一些注意事项：

**作用域**： 对于 useBean 而言，如果 scope 字段省略，则默认为 page 作用域。一共有 application、request、session、page 四个作用域。如果 type 字段省略，那么将会用 class 字段的类型来设置引用的类型。如果 class 字段省略，那么将会使用 type 的类型来获取对象并且转型且使用此类型作为引用。不能同时省略 class 和 type。

**引用和实例的类型**： 对于 useBean，还有一个有趣的细节，如果它从给定范围找不到这个对象，那么它会自己创建一个对象，并且放置在 page 作用域中。这对于下一节JSP直接处理请求的情况下较为常用。尤其需要注意，对于创建对象而言，如果不指定 class，而只有 type，则不会自动创建对象。**即 type 只用于转型和定义引用类型， class 用于创建不存在的Beans。因为获取对象不需要知道其本身的类型**，因此对于使用Bean，而不是创建，没有class，只有type也可以工作。

**属性存取动作的注意事项** ：name 指的是我们在本地定义的引用， property 是我们想要获取的属性，value 是我们希望赋给的值。此外，还可以有 param 字段，在下一节介绍。

**对于Java Beans的要求**: 使用标准动作，对于 Java Beans 有要求：必须符合 Beans 的 Property 命名规则，比如 name 属性必须有 getName 和 setName 两个公开可访问的方法。这个 name 可以是 private 的。此外，**必须有无参 public 的构造器**。其次，Beans 的类不能是 abstract 或者 interface，并且可公开访问。

## 1.2 创建 Beans 

对于下列情况：直接从请求到达 jsp 文件，而不经过 servlet，那么，我们就要在 JSP 中构建 Beans：

```html
<form action="/helloagain" method="post">
    Name: <input type="text" name="name" /> <br/>
    School: <input type="text" name="school" /> <br/>
    Age: <input type="text" name="age" /> <br/><br/><br/>
    <input type="submit" />
</form>
```

在 `/helloagain` URL 映射下的 helloagain.jsp 文件代码片段：

```jsp
<jsp:useBean id="user" class="com.mazhangjing.model.Student" type="com.mazhangjing.model.Person">
    <jsp:setProperty name="student" property="name" value="<%= request.getParameter(\"name\") %>"/>
    <jsp:setProperty name="student" property="address" param="school" />
    <jsp:setProperty name="student" property="age" />
    <jsp:setProperty name="student" property="*"/>
</jsp:useBean>

<h1>Hello, <jsp:getProperty name="student" property="name"/>.</h1>
<p>You come from <jsp:getProperty name="student" property="address"/>.
And you are <jsp:getProperty name="student" property="age" /> now.</p>
```

可以看到，区别在于，useBean 因为需要创建对象，所以 class 字段是不可少的，type 字段可以指定或者不指定，这取决于你是否想要转型以及使用更为抽象类型的引用。如果不写 scope，那么默认是在 pageScope。

**useBean 内部的 setProperty 详谈：** 对于 setProperty 而言，之前我们说过，常用的是 name、property 以及 value 这三个字段，对于 value，我们可以使用脚本从 param 中获得，但是这样很丑，双引号需要转义（这也说明了脚本先处理，然后才是标准动作）。但是，我们可以直接指定 param 字段，从 param 中获取值。如果 param 和 property 是一致的，那么甚至不用写 param，程序会自动处理。此外，如果所有的 property 和 param 都是一致的（proerpty >= param），那么使用星号可以直接建立两者映射。

`jsp：` 本身就是一个 xml 规范，因此 useBean 中间不能有多余的注释，否则无法通过 xml 审查。

对于单纯的获取 Bean 而不是新建 Bean，这里面的属性设置代码将不会被调用！！！！！也就是说，如果 useBean 得到了 Bean，那么直接返回这个 Bean，其包裹的 setProperty 代码不会调用，除非是它找不到Bean，那么会调用这些设置属性的内部代码，然后再返回 Bean。但是，对于放置在外部的 setProperty，不论 Bean 是否存在，因为代码执行到外部后，一定有一个 Bean，所以一定会被调用并且起作用的。

**Java Beans 值类型的转换问题：** String和基本格式可以按照Bean的需求来转换，比如Bean需要int，那么setProperty会自动转换值为int，然后进行设置。但是，如果value掺杂了脚本，比如name字段那样，String不会自动转换成为int。对于非这些类型，则会强制使用toString得到的结果作为参数传递。

如果需要对于参数进行检查，或者进行 parseInt 之类的东西，最好使用 Servlet 进行处理，然后 forward 到 JSP 页面。JSP页面应该只用于View层，不参与逻辑计算。（当然，说这句话的意思是很多人为了不使用Servlet，直接在JSP上面写一段脚本进行参数检查，这样唯一的优点是jsp的默认映射不用写。而按照标准写法，jsp映射应当独立定义，如果按照标注的话，还不如映射到servlet，然后再转jsp结构清晰，容易修改。）

标准动作能够解决的事情很有限，比如我有个 dog 的属性，是一个 Dog 的类型，我想要获取 Dog 的 name 属性，不能写： mydog.name 。这时就需要 EL 出场了，EL 叫做表达式语言，专门被设计用来从映射或者多层的Bean中获取属性并且进行简单的逻辑计算。

## 1.3 JSP 片段包含

对于 Web 开发而言，一个页面中有一些元素是固定不变的，比如页脚和页眉，使用 `<jsp:include page="xx.jsp">` 动作或者 `<%@ include file="xx.jsp"%>` 指令进行包含。

对于命令而言，当请求到这个页面的时候，容器会自动将这两个JSP文件转换成为一个 Servlet 类进行响应。标准没有规定说当文件变化时容器需不需要动态更新这个Servlet。但是 TOMCAT 可以做到动态更新。

对于动作而言，当请求到这个页面的时候，容器会分别到这两个JSP文件对应的两个Servlet中（通过请求分发）执行相应代码。效率不如命令高。对于纯粹静态，且不需要更改的东西，使用命令更好。如果需要动态更改，比如当文件改变后容器可以自动识别（容器会自动识别JSP文件的更改，然后动态更新其Servlet，这个是标准规定的）。此外，使用动作还可以做到向这些片段中传递参数。比如：

```jsp
<jsp:include page="footer.jsp" flush="true">
    <jsp:param name="currentYear" value="2018"/>
    <jsp:param name="currentMonth" value="09" />
</jsp:include>
```

以下是被包含的一个片段：

```jsp
<%@taglib prefix="server" uri="RollMe" %>
<p>Welcome to my wibesite, current time is ${server:time()}。 ${param.currentYear}</p>
```

**jsp和jspf的棘手问题：** 注意这里的 param 动作，如果 include 的是 footer.jspf, 那么将无法执行 `param.currentYear` 这句EL。只会将其打印出来。

按理来说，对于 EL 的解析发生在 Servlet 脚本之后，发生在 jsp 指令之前。不知道为何，对于jspf的包含指令提前发生了，而这时EL尚未对于jspf进行解析。或者说，在Tomcat9中，EL不对jspf后缀的文件单独解析，仅对jsp文件解析，然后交给jsp指令继续解析。

## 1.4 JSP 派发

不建议以任何理由使用 JSP 的派发，但是 JSP 提供了这一支持：

`<jsp:forward page="handleIt.jsp" />` 和 `req.getRequestDispatcher("/hello.jsp").forward(req,resp);` 有异曲同工之妙。

需要注意，派发前的 HTML 都是缓存，尚未写入到 out 中，如果进行派发，那么这些缓存将会被清除。如果在派发前调用了 out.flush() 那么将派发失败，对于用户而言，他将会看到缓存的输出，而不是派发的页面。

需要注意，对于 `<jsp:include page="footer.jsp" flush="true">` 这句话，flush 对 out 进行了 flush，因此放在其后会导致派发失败。



# 2. EL 表达式语言

## 2.1 语法介绍

表达式语言 EL 是一种在JSP页面中使用的语法，基本结构为 `${a.b.c}` 或者 `${a["Hello"]}`。这种语法主要是为了从 映射、容器、Bean 中获取属性以及其嵌套属性的，其本身不参与 Beans 的创建过程。只能够给定的 映射、容器、Bean 中获取值。

**字母含义：** 其中对于a而言，它可以是隐式对象，也可以是各个作用域的属性（如果不指定作用域对象，则从page开始搜索，找到即返回）。

**点号操作符：** 点号是一种操作符，这个操作符可以用来访问 a 中的一些东西。使用点号操作符的要求是：点号左边必须是 MAP映射（各个作用域的键值对）或者Bean，点号右边可以是 MAP 映射的 Key，也可以是 Bean 的属性。并且，点号右边不能够是Java不支持的属性字段，比如内置关键字，数字开头的变量名、包含横杠的变量 `accept-language`。即便这个数字可能是左边一个映射的 Key，而不是 Bean 的属性也不行。

**中括号操作符：** 改进方法是使用[]操作符，所有在这个操作符中的东西都会被强制转换或者计算成为字符串，然后再进行操作。`${a["Hello"]}` 其中 a 表示映射或者 Bean，Hello 字段表示映射的 Key 或者 Bean 的属性。方括号操作符就可以满足 `${a["com.me"]}` 这样的需求，因为 `a.com.me` 会被认为是 a 的 com 属性/Key 的 me 属性/Key。

方括号操作符的左边不仅仅支持 MAP 和 Beans，现在还支持 List、原生数组。对于这些没有Key的对象的访问，使用下标，下标从 0 开始，可以是 int 类型，或者是 String 类型，String 类型的话会被自动转为 int 类型，然后取其元素。对于 Map 和 Beans，所有的非字符串字面量的输入都会被进行计算，然后转换成为字符串字面量取Key或者Beans的属性值。

最后，中括号可以嵌套，`${a[b[c[0]]]}`。尤其需要注意，所有能够用点号操作符操作的都可以用中括号操作符操作。此外，所有含有点的、横线的 Key 都必须使用中括号操作符操作。比如 “a.b” 或者 “accept-language” 这些字段。

## 2.2 EL 隐式变量

EL 可以从哪些地方获取 Map、List、Array、Bean呢？

- pageScope, requestScope, sessionScope, applicationScope 这是四个作用域。这些作用域指的是这些作用域的 Attribute 字段，也就是那些被 `r/p/s/a.setAttritube("id",value);` 过的对象，才可以用 `r/p/s/a.id` 获取。


- param，paramValues 这是直接向 JSP 发送给请求，而不经过Servlet时的参数以及其作用域。注意pValues的使用方法，Key要加双引号。比如 `paramValues["id"][0]`


- header,headerValues 这是 Header 的参数词典。之所以有两个的原因是后者代表了相同 key 的多个 value，使用 headerValues["header_name"][0] 来取出。


- cookie 注意，不是cookies


- initParam 上下文初始化参数，注意，这里不是Servlet的初始化参数！！！是APP context 的参数


- pageContext 一个Bean，其实是唯一的一个EL 直接访问的 Bean，其余上面的都是映射。通过它可以获得一些奇怪的东西，比如请求的方法 `pageContext.request`。需要区分 requestScope 和 pageContext.request 前者是一个字典，保存的是 request setAttritube 的东西，后者是一个 Beans。

此外，一个尤其容易出错的点是：使用脚本在 page 本地声明的引用，不能直接被 EL 调用。如果使用脚本，必须将这个变量传递给 page，使用 `pageContext.setAttritube("id",value);` 后，才可以 `pageScope.id` 这样使用。其中原因可能是 脚本在 Servlet 中进行定义，但是 EL 的解析并不在同一个类中进行。

## 2.3 示例代码

下面展示了对于不同 Scope 设置 Attritube 后，使用 EL 的代码：

```java
request.setAttribute("name","Corkine Ma");

ArrayList group = new ArrayList<>();
group.add("Hello"); group.add("Meet");
request.setAttribute("group",group);

Person[] people = new Person[5];
people[0] = new Person("P1",20,"P1Address");
people[1] = new Person("P2",22,"P2Address");
people[2] = new Person("P3",23,"P3Address");
pageContext.setAttribute("people",people);
```

使用标准动作，默认是 pageScope。

```jsp
<jsp:useBean id="student" class="com.mazhangjing.model.Student" type="com.mazhangjing.model.Person">
    <jsp:setProperty name="student" property="name" value="Marvin" />
    <jsp:setProperty name="student" property="address" value="Central China Normal University" />
</jsp:useBean>
```

**对于映射：** 我们使用一下 RequestScope 这个映射中 name 的 Key 获取： `${requestScope.name}`。 换一种写法,使用中括号的结果是： `${requestScope["name"]}`

**对于Beans：** 现在我们使用一下 Bean 的属性获取： `${student.name}`, 这个学生的地址是 `${student.address}`。 同样的，换一种写法，使用中括号而不是点号操作符：`${student["name"]}` `${student["address"]}。`

**对于列表：** 对于List和数组，只能使用中括号，但是相比较标准标记，我们可以访问多层对象了。列表下标1中的元素是： `${group}`。 列表下标越界后的结果是： `${group[100]}`。 列表下标用字面量取下标1的结果是： `${group["1"]}`

**对于数组：** 数组也可以，这是一个保存了 Person 对象的数组,我现在访问它以及它的属性： `${people[1]}`，`${people[1]["address"]}` 混用两种操作符也可以： `${people["1"].address}`

注意，所有的这些 Bean 或者映射都必须在各个作用域的 Attribute 中能够找到。对于 pageScrop，不能够单纯在JSP的Servlet中定义变量，必须交给 pageContext.setAttribute 后才可以访问。标准标记比较特殊，它会自动添加到相应的Scope中，如果不写的话，默认是pageScope。


使用 EL 访问隐式变量的示例代码如下：

```jsp
First Choose Language: ${headerValues["accept-language"][0]}
Cookie: ${cookie}  注意，cookie不是cookies
Param: ${param}
ParamName: ${paramValues["name"][0]}
InitParam: ${initParam["server"]}
Method through pageContext Bean : ${pageContext.request.method}
```
对于 pageContext，这是一个Bean，不是一个映射。

对于XXValues这种，第一个参数必须是Key，第二个参数必须是下标，缺一不可。下标为0等同于 XX 不加 Values 方法。

此外，需要注意的是，EL表达式错误不像标记那样，如果结果为null不会报错，但是如果语法错误，则直接不生成文档。

此外，需要注意的是，initParam 是 application context，不是 servlet init-param。

## 2.4 EL 运算操作符

EL支持一些操作符，比如：加减乘除和与或非、等于、大于、小于、大于等于、小于等于。可以使用JAVA的运算符或者是字母简写：equal - eq, less equal - le 等。逻辑运算可以用JAVA原生或者是Python类型的操作符。

算术运算如下：
```
${2 >= 1} ${2 == 1} ${2 < 1} ${2 gt 1} ${2 eq 1} ${2 le 1}

${2 + 1 - 3 / 4 * 5}

${4/0} ${4 mod 2} ${4 % 2} 注意，除0得无限
```

逻辑运算如下：

```

${false && true}
${true and false}
${true || false}
${2 != 7 or false}
${!true}
${not true} 
注意 ${1 || 3} 这样的写法会直接报错！！！
```

## 2.5 EL 函数调用

我们现在几乎完全摆脱了脚本：使用标准动作来创建和简单调用其属性并展示，使用EL表达式语言来更高级的获取Bean或者映射的属性并展示，此外还能进行一些简单的操作符运算。最后，我们还想要有在 JSP 中调用函数的能力，这个时候就要使用 TLD 了。

TLD 需要创建一个以 .tld 结尾的文件，此文件应可以被容器找到。容器会自动确认此后缀文件，不用告诉容器它在哪里。示例如下：

```xml
<uri>RollMe</uri>
<function>
    <name>roll</name>
    <function-class>com.mazhangjing.model.DiceRoller</function-class>
    <function-signature>int rollIt()</function-signature>
</function>
<function>
    <name>time</name>
    <function-class>com.mazhangjing.model.DiceRoller</function-class>
    <function-signature>java.lang.String getTime()</function-signature>
</function>
```

其中，uri指的是此 tld 文件的位置， function指的是此tld的一个函数，其中需要定义name和函数映射的类以及签名。

被调用的方法如下：

```java
public class DiceRoller  {
    public static int rollIt() {
        return (int)Math.random() * 6 + 1;
    }
    public static String getTime() {
        return new Date().toString();
    }
}
```

可以看到，此方法只和 tld 文件中的 function-class 以及 function-signature 有关。而我们的调用指令只和 tld 的 uri 以及 function 的 name 有关。因为 tld 文件动态加载，因此很好的将JSP和Java原生代码进行了解耦。

注意，签名需要提供返回值以及方法名称，甚至包括传入参数。在使用 taglib 的时候，和 XML 的用法很相似，定义一个前缀，来代替 uri，然后调用 前缀:方法() 即可。

而对于此 tld 文件的用法如下：

```jsp
<%@taglib prefix="mine" uri="RollMe" %>
${mine:roll()}
```

这样el就可以通过taglib指令就可以调用原生JAVA方法了。

因此，通过 Java class ==> tld file ==> taglib define ==> EL call 的流程，我们可以在 EL 中调用 Java 的原生方法。这其实很简单，其中 mine：的含义和 jsp: 的含义接近，都是指的一个命名空间，区别在于，jsp的空间是隐含的，而我们自己的空间需要使用 taglib 指定，这通过 taglib 指令来指定，其中这个指令需要指定一个 uri，也就是一个 tld 文件，这个文件规定了可供调用的方法，以及这个方法是如何被特定的 JAVA 对象调用的。通过在 EL 中使用这个命名空间下的方法，就可以完成对于 JAVA 类的调用。

命名空间本身是 XML 为了规范标签调用而创建的，在这里作为 taglib 的规范，指定了 EL 能够调用的 tld 这个元文件中定义好的标签（方法）。

## 2.6 EL 配置

需要注意: EL可以在 DD 中或者在 page 指令中设置禁用；

```xml
<jsp-config>
    <jsp-property-group>
        <url-pattern>*.jsp</url-pattern>
        <el-ignored>false</el-ignored>
        <trim-directive-whitespaces>true</trim-directive-whitespaces>
    </jsp-property-group>
</jsp-config>
```

或者 `<%@page isELIgnored="true" %>`